# Chat Models - 聊天模型接口详解


## 概述

Chat Models 是 LangChain 中用于与聊天模型交互的核心接口。它们实现了 `BaseChatModel` 接口，同时也实现了 `Runnable` 接口，支持标准的流式处理、批量处理等功能。

### 核心特性

- **消息格式**: 接受消息列表作为输入，返回消息作为输出
- **角色支持**: 支持 `system`、`human`、`assistant` 等角色
- **流式处理**: 支持实时流式输出
- **批量处理**: 支持批量请求优化
- **工具绑定**: 支持绑定外部工具
- **结构化输出**: 支持结构化数据输出

### 关键方法

1. **`invoke`** - 主要交互方法
2. **`stream`** - 流式输出
3. **`batch`** - 批量处理
4. **`bindTools`** - 绑定工具
5. **`withStructuredOutput`** - 结构化输出

## LangChain Chat Models 完整示例

In [1]:
"""
LangChain 0.3 Chat Models 完整示例
"""
# 引入依赖
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from typing import List, Dict, Any
import asyncio


### 1. 基础聊天模型初始化

In [3]:

def basic_chat_model():
    """基础聊天模型使用"""
    print("=" * 50)
    print("1. 基础聊天模型使用")
    print("=" * 50)

    # 初始化模型
    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b",
        temperature=0.7
    )

    # 单条消息
    response = model.invoke([HumanMessage(content="你好，介绍一下自己")])
    print(f"AI回复: {response.content}")
    print(f"响应元数据: {response.response_metadata}")

    # 多轮对话
    messages = [
        SystemMessage(content="你是一个友好的AI助手"),
        HumanMessage(content="我叫张三"),
        AIMessage(content="你好张三！很高兴认识你。"),
        HumanMessage(content="我的名字是什么？")
    ]

    response = model.invoke(messages)
    print(f"\n多轮对话回复: {response.content}")
basic_chat_model()

1. 基础聊天模型使用
AI回复: 你好！我叫Qwen，是阿里巴巴推出的一个预训练模型。我是由阿里云研发的一系列超大规模语言模型之一，旨在通过理解和模拟人类语言来提供帮助和支持。

作为AI助手，我的能力包括但不限于回答问题、讲故事、提供建议和解决问题等。我可以生成文本内容，比如撰写故事、新闻文章或诗歌。此外，我也能参与对话并理解复杂的指令和幽默感。

我被设计为在多种情境下都能提供有用的信息，并且随着更多的训练数据和支持系统的增强，我能够更好地理解和回应人类的需求。不过，请注意，尽管我在许多任务上都可以提供帮助，但在某些专业领域可能无法达到专家水平。总的来说，我的目标是成为一个友好、可靠并且尽可能满足用户需求的伙伴。
响应元数据: {'model': 'qwen2.5:3b', 'created_at': '2025-07-22T08:06:14.515785Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9856646869, 'load_duration': 41191128, 'prompt_eval_count': 33, 'prompt_eval_duration': 369625763, 'eval_count': 159, 'eval_duration': 9441715871, 'model_name': 'qwen2.5:3b'}

多轮对话回复: 你的名字是张三。


### 2. 流式处理示例

In [5]:
# 2. 流式处理示例
def streaming_chat():
    """流式处理示例"""
    print("\n" + "=" * 50)
    print("2. 流式处理示例")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    messages = [HumanMessage(content="请写一首关于春天的诗")]

    print("流式输出:")
    for chunk in model.stream(messages):
        print(chunk.content, end="", flush=True)
    print("\n")
streaming_chat()


2. 流式处理示例
流式输出:
春风如故人，轻轻绕我身。
万物皆复苏，欣欣向阳新。

桃花笑东风，柳絮舞轻尘。
人间四月天，春色正浓时。 

此情此景里，心自安处静。
只愿时光长，共赏这美好。



### 3. 批量处理示例

In [6]:
# 3. 批量处理示例
def batch_processing():
    """批量处理示例"""
    print("\n" + "=" * 50)
    print("3. 批量处理示例")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 批量请求
    batch_messages = [
        [HumanMessage(content="1+1等于多少？")],
        [HumanMessage(content="Python是什么？")],
        [HumanMessage(content="解释一下机器学习")]
    ]

    responses = model.batch(batch_messages)

    for i, response in enumerate(responses):
        print(f"问题{i+1}回复: {response.content[:50]}...")
batch_processing()


3. 批量处理示例
问题1回复: 1+1等于2。...
问题2回复: Python是一种高级编程语言，由吉米·赫茨和吉尔伯特·格雷在1989年发明。Python的设计哲学...
问题3回复: 机器学习是一种人工智能的子领域，它的目标是设计和开发算法和统计模型，使计算机能够从数据中自动获取知识...


### 4. 使用提示模板

In [ ]:

# 4. 使用提示模板
def with_prompt_template():
    """使用提示模板"""
    print("\n" + "=" * 50)
    print("4. 使用提示模板")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 创建提示模板
    prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一个{role}，请用{style}的风格回答问题"),
        ("human", "{question}")
    ])

    # 创建链
    chain = prompt | model | StrOutputParser()

    # 调用
    response = chain.invoke({
        "role": "诗人",
        "style": "古典",
        "question": "描述一下秋天的景色"
    })

    print(f"诗人回复: {response}")
with_prompt_template()

### 5. 带记忆的对话

In [ ]:

# 5. 带记忆的对话
def conversation_with_memory():
    """带记忆的对话"""
    print("\n" + "=" * 50)
    print("5. 带记忆的对话")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 对话历史
    conversation_history = []

    def chat_with_memory(user_input: str) -> str:
        # 添加用户消息
        conversation_history.append(HumanMessage(content=user_input))

        # 调用模型
        response = model.invoke(conversation_history)

        # 添加AI回复到历史
        conversation_history.append(response)

        return response.content

    # 模拟对话
    print("用户: 我是一名程序员")
    ai_response = chat_with_memory("我是一名程序员")
    print(f"AI: {ai_response}")

    print("\n用户: 我主要使用什么编程语言？")
    ai_response = chat_with_memory("我主要使用什么编程语言？")
    print(f"AI: {ai_response}")
conversation_with_memory()

### 6. 异步处理示例

In [ ]:

# 6. 异步处理示例
async def async_chat():
    """异步处理示例"""
    print("\n" + "=" * 50)
    print("6. 异步处理示例")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 异步调用
    response = await model.ainvoke([
        HumanMessage(content="异步调用测试")
    ])
    print(f"异步回复: {response.content}")

    # 异步流式处理
    print("\n异步流式输出:")
    async for chunk in model.astream([
        HumanMessage(content="请简单介绍Python")
    ]):
        print(chunk.content, end="", flush=True)
    print("\n")

### 7. 多模型比较

In [ ]:

# 7. 多模型比较
def multi_model_comparison():
    """多模型比较"""
    print("\n" + "=" * 50)
    print("7. 多模型比较")
    print("=" * 50)

    # 不同模型
    models = {
        "qwen2.5:3b": ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b"),
        "qwen2.5:0.5b": ChatOllama(base_url="http://localhost:11434", model="qwen2.5:0.5b")
    }

    question = [HumanMessage(content="什么是人工智能？")]

    for model_name, model in models.items():
        try:
            response = model.invoke(question)
            print(f"\n{model_name}回复:")
            print(response.content[:100] + "...")
        except Exception as e:
            print(f"{model_name}调用失败: {e}")

### 8. 带参数配置的模型

In [ ]:

# 8. 带参数配置的模型
def model_with_parameters():
    """带参数配置的模型"""
    print("\n" + "=" * 50)
    print("8. 带参数配置的模型")
    print("=" * 50)

    # 不同温度设置
    temperatures = [0.1, 0.7, 1.0]

    for temp in temperatures:
        model = ChatOllama(
            base_url="http://localhost:11434",
            model="qwen2.5:3b",
            temperature=temp
        )

        response = model.invoke([
            HumanMessage(content="创造一个有趣的故事开头")
        ])

        print(f"\n温度{temp}的回复:")
        print(response.content[:80] + "...")

### 9. 错误处理示例

In [ ]:

# 9. 错误处理示例
def error_handling():
    """错误处理示例"""
    print("\n" + "=" * 50)
    print("9. 错误处理示例")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    try:
        # 正常调用
        response = model.invoke([HumanMessage(content="测试")])
        print(f"成功: {response.content[:50]}...")

        # 模拟错误（使用不存在的模型）
        error_model = ChatOllama(
            base_url="http://localhost:11434",
            model="nonexistent-model"
        )
        error_response = error_model.invoke([HumanMessage(content="测试")])

    except Exception as e:
        print(f"捕获错误: {type(e).__name__}: {e}")

### 10. 自定义输出解析

In [ ]:

# 10. 自定义输出解析
def custom_output_parsing():
    """自定义输出解析"""
    print("\n" + "=" * 50)
    print("10. 自定义输出解析")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 自定义解析器
    class CustomParser:
        def parse(self, text: str) -> Dict[str, Any]:
            return {
                "content": text,
                "length": len(text),
                "word_count": len(text.split())
            }

    parser = CustomParser()

    # 创建链
    def custom_chain(input_text: str):
        response = model.invoke([HumanMessage(content=input_text)])
        return parser.parse(response.content)

    result = custom_chain("请简单介绍一下自己")
    print(f"解析结果: {result}")

In [2]:

# 主函数
def main():
    """运行所有示例"""
    print("LangChain 0.3 Chat Models 完整示例")
    print("=" * 60)

    # 运行所有示例
    basic_chat_model()
    streaming_chat()
    batch_processing()
    with_prompt_template()
    conversation_with_memory()
    multi_model_comparison()
    model_with_parameters()
    error_handling()
    custom_output_parsing()

    # 异步示例需要单独运行
    print("\n运行异步示例...")
    asyncio.run(async_chat())

if __name__ == "__main__":
    main()

LangChain 0.3 Chat Models 完整示例
1. 基础聊天模型使用
AI回复: 你好！我叫Qwen，是阿里云开发的一款超大规模语言模型。我是通过深度学习技术训练而成的，可以生成各种类型的文本内容，比如故事、诗歌、新闻报道等，还能回答关于历史、地理、科学和各种主题的问题。此外，我也能够帮助你完成许多任务，例如编写邮件、制作策划方案等。我还可以根据你的需求创作音乐或艺术作品。

总之，我的目的是尽可能多地了解世界，并以多种方式与你进行互动，以便更好地服务您。不过，请注意虽然我能生成多样化的文本内容，但我并不具备任何超越自身功能的超能力。同时，我也能保护用户的隐私和安全，确保所有的交流都是在合法、合规的前提下进行的。

如果您有任何问题或需要帮助的地方，欢迎随时提问。
响应元数据: {'model': 'qwen2.5:3b', 'created_at': '2025-07-22T06:19:28.561232Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 11466729520, 'load_duration': 35680223, 'prompt_eval_count': 33, 'prompt_eval_duration': 325998399, 'eval_count': 164, 'eval_duration': 11104060353}

多轮对话回复: 你的名字是张三。有什么我可以帮助你的吗？

2. 流式处理示例
流式输出:
春风轻拂绿柳枝，  
细雨绵绵润芳菲。  
桃花笑开映朝霞，  
万物欣欣向荣华。

鸟鸣山谷间悠扬，  
蝴蝶翩飞花丛中。  
春光明媚驱寒去，  
人间处处是画卷。


3. 批量处理示例
问题1回复: 1+1等于2。...
问题2回复: Python是一种高级编程语言，由Guido van Rossum在1980年代末开发，并于1991...
问题3回复: 机器学习是一种人工智能技术，它使计算机能够在不进行明确编程的情况下从数据中自动学习并改进其性能。换句...

4. 使用提示模板
诗人回复: 秋光染红枫叶，黄花遍地铺。
残阳如血照千里，清风徐

KeyboardInterrupt: 

## 通用初始化方法

In [7]:
"""
使用 initChatModel 通用初始化方法
"""
from langchain.chat_models import init_chat_model

# 通用初始化 - 支持多种模型提供商
def universal_init_examples():
    """通用初始化示例"""

    # OpenAI
    gpt4o = init_chat_model("gpt-4o", {
        "model_provider": "openai",
        "temperature": 0
    })

    # Anthropic
    claude = init_chat_model("claude-3-opus-20240229", {
        "model_provider": "anthropic",
        "temperature": 0
    })

    # Google Vertex AI
    gemini = init_chat_model("gemini-1.5-pro", {
        "model_provider": "google-vertexai",
        "temperature": 0
    })

    # 也可以在模型名中指定提供商
    claude_alt = init_chat_model("anthropic:claude-3-opus-20240229", {
        "temperature": 0
    })

    # 统一接口调用
    models = [gpt4o, claude, gemini]
    for model in models:
        try:
            response = model.invoke("你好")
            print(f"{model.__class__.__name__}: {response.content}")
        except Exception as e:
            print(f"模型调用失败: {e}")

if __name__ == "__main__":
    universal_init_examples()

TypeError: init_chat_model() takes from 0 to 1 positional arguments but 2 were given

## 高级功能

In [8]:
"""
Chat Models 高级功能示例
"""
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
import json

def advanced_features_demo():
    """高级功能演示"""

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 1. 并行处理
    parallel_chain = RunnableParallel({
        "translation": ChatPromptTemplate.from_template("翻译: {text}") | model | StrOutputParser(),
        "summary": ChatPromptTemplate.from_template("总结: {text}") | model | StrOutputParser(),
        "sentiment": ChatPromptTemplate.from_template("情感分析: {text}") | model | StrOutputParser()
    })

    result = parallel_chain.invoke({"text": "今天天气真好，我很开心"})
    print("并行处理结果:")
    for key, value in result.items():
        print(f"{key}: {value[:50]}...")

    # 2. 条件分支
    def route_question(input_dict):
        question = input_dict["question"]
        if "数学" in question or "计算" in question:
            return "math"
        elif "翻译" in question:
            return "translation"
        else:
            return "general"

    # 3. 自定义处理链
    def create_specialized_chain(question_type):
        if question_type == "math":
            prompt = "作为数学专家，请解答: {question}"
        elif question_type == "translation":
            prompt = "作为翻译专家，请翻译: {question}"
        else:
            prompt = "请回答: {question}"

        return ChatPromptTemplate.from_template(prompt) | model | StrOutputParser()

    # 测试路由
    questions = [
        "1+1等于多少？",
        "请翻译'Hello'",
        "什么是人工智能？"
    ]

    for q in questions:
        route = route_question({"question": q})
        chain = create_specialized_chain(route)
        response = chain.invoke({"question": q})
        print(f"\n问题类型: {route}")
        print(f"问题: {q}")
        print(f"回答: {response[:100]}...")

if __name__ == "__main__":
    advanced_features_demo()

并行处理结果:
translation: 今天的天气真是太好了，我感到非常高兴。...
summary: 今天的天气非常好，这让您感到很开心。这正是美好的一天的缩影。...
sentiment: 这条信息的情感倾向是积极的。句子表达了说话者对今天的天气感到满意，并因此感到开心。...

问题类型: general
问题: 1+1等于多少？
回答: 1+1等于2。...

问题类型: translation
问题: 请翻译'Hello'
回答: 'Hello' 在英文中直接对应 '你好' 在中文中。所以它的翻译是 '你好'。...

问题类型: general
问题: 什么是人工智能？
回答: 人工智能（Artificial Intelligence，简称AI）是指由人设计出的一套处理信息的系统或者算法。这些系统或算法能完成一些通常需要人类智能才能执行的任务，比如视觉识别、语音识别、决策制定...


## 关键特性总结
1. 统一接口: 所有聊天模型都实现相同的接口
2. 消息驱动: 基于消息列表的交互模式
3. 流式支持: 实时流式输出能力
4. 批量优化: 支持批量请求提高效率
5. 异步支持: 完整的异步操作支持
6. 可组合性: 与其他LangChain组件无缝集成
7. 标准化参数: 温度、最大令牌等标准参数
8. 错误处理: 完善的异常处理机制
这些示例展示了 LangChain 0.3 中 Chat Models 的核心功能和使用方法，可以根据具体需求选择合适的模式进行开发